<a href="https://colab.research.google.com/github/mikedparrott/micro-lessons/blob/master/Connect_to_AWS_Database_Using_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Micro Course:
##**Create an SQL Database & Connect to it with R**

Use the following notebooks to build an SQL database on AWS and connect to it with R. See the [micro lessons github repo](https://github.com/mikedparrott/micro-lessons) for other useful micro lessons.
- [Create an SQL Database on Amazon Web Services](https://github.com/mikedparrott/micro-lessons/blob/master/Create_an_SQL_Database_on_Amazon_Web_Services.ipynb)
- [Connect to AWS Database Using R](https://github.com/mikedparrott/micro-lessons/blob/master/Connect_to_AWS_Database_Using_R.ipynb)
- [Delete AWS Database](/notebooks/mlcc/intro_to_sparse_data_and_embeddings.ipynb)


---

## Micro Lesson II. Connect to AWS Database Using R

###1.   We will rely on the dplyr and dbplyr packages to connect to our database. 
First, install and then load the dplyr and dbplyr packages 

**Code:**
```
# Install the dplyr and dbplyr libraries.

install.packages("dplyr")
install.packages("dbplyr")

# Load the dplyr and dbplyr libraries.
library(dplyr)
library(dbplyr)
```


###2.   Load your database endpoint, port, username, and password into R objects.
Here we are setting up the data for different arguments in the dbplyr function we will use to connect to our RDS database.

**Code:**
```
host="endpoint listed in RDS"
port=3306
dbname="your_database_name"
user="your_username"
password="your_password"
```

###3.   Connect to your MySQL database using the src_mysql() function from the dbplyr package.
The dbplyr package has several functoins to connect to different types of SQL driven relational databases.  We use src_mysql to connect to our MySQL database.

**Code:**
```
# Connect to database.
my_db=src_mysql(dbname=dbname,host=host,port=port,user=user,password=password)

#see ?src_mysql for other types of database connections
```

###4.   Load a table into your database.
You can copy an R data frame to your AWS database using the following code.  

**Code:**
```
#Adding example mtcars dataset to database. 

# Look at first six observations of mtcars data
head(mtcars)

# Copy mtcars data to a table in your database
copy_to(my_db,mtcars,temporary = FALSE) # Use overwrite=TRUE to replace an existing table

#Now, let's use the src_tbls() function check if the table has been transferred.

src_tbls(my_db)
```


###5.   Interact with your MySQL database table in the cloud using dbplyr and dplyr.
We have transfered the data. Now, we can query agaist the database. To connect to a table, we use the tbl function. We use the same dplyr verbs that
we use for typical data manipulation to work with databases .  The dbplyr library translates the R code we write using dplyr verbs to SQL code.

**Code:**
```
mtcars_db = tbl(my_db,"mtcars") # Connect to table in database for queries
class(mtcars_db)
 
head(mtcars_db) #Notice the source/ Database connection.  We are interacting with data on RDS.  

filter(mtcars_db, cyl == 8, qsec > 16) #Filter database in r

# Notice that these are lazy queries.  To actually return the data from your query from RDS run collect() after assigning your query to a new object

querydata<-filter(mtcars_db, cyl == 8, qsec > 16) #filter database in r

#need to run collect to return data from query into your R session

finaldata<-collect(filter(mtcars_db, cyl == 8, qsec > 16)) 

# Another query example that returns data into R session

finaldata<-collect(select(mtcars_db, mpg, wt)) 
```


###6.   Have a look at the SQL that dbplyr is using to generate your queries
Dbplyr takes dplyr code and translates it into SQL queries.  We can print out the raw SQL code using show_quer
**Code:**
```

#print query translation
show_query(filter(mtcars_db, cyl == 8, qsec > 16) ) #dbplyr converts dplyr verbs to sql
                                                    #then the sql is sent to RDS to extract data

```



###7.   Or simply connect directly to your database and send your own SQL queries.

**Code:**
```
#You can also connect using DBI and send raw sql
library(DBI)
library(RMySQL)

conn <- dbConnect(
  drv = RMySQL::MySQL(),
  dbname=dbname,host=host,port=port,user=user,password=password)

dbGetQuery(conn, "SELECT * FROM mtcars LIMIT 5;")
```







  
